In [48]:
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

# plt.style.use('ggplot')

# Import

First we open a file containing a dictionary of columns and associated data types generated in `../../optimize/`, then we read input file.

In [49]:
with open('../input/dtypes.yaml', 'r') as yamlfile:
    column_types = yaml.load(yamlfile)

read_csv_opts = {'sep': '|',
                 'quotechar': '"',
                 'compression': 'gzip',
                 'encoding': 'utf-8',
                 'dtype': column_types,
                 'parse_dates': ['MissionDate'],
                 'infer_datetime_format': True}

df = pd.read_csv('../input/ice-air.csv.gz', **read_csv_opts)

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732625 entries, 0 to 1732624
Data columns (total 48 columns):
Status                          category
Sex                             category
Convictions                     category
GangMember                      category
ClassLvl                        float32
Age                             float32
MissionDate                     datetime64[ns]
MissionNumber                   uint32
PULOC                           category
DropLoc                         category
StrikeFromList                  float32
ReasonStruck                    category
R-T                             category
Code                            category
CountryOfCitizenship            category
Juvenile                        bool
MissionWeek                     uint8
MissionQuarter                  uint8
MissionYear                     uint16
MissionMonth                    uint8
Criminality                     category
FamilyUnitFlag                  float32


In [50]:
with open('../input/airport_dict.yaml', 'r') as yamlfile:
    airport_dict = yaml.load(yamlfile)

# Status

In [51]:
removals = df['R-T'] == 'R'
transfers = df['R-T'] == 'T'

In [52]:
status_count = df[removals].drop_duplicates(subset='AlienMasterID')['Status'].str.upper().value_counts(dropna=False)

In [53]:
status = pd.read_csv('../../share/resources/status.csv')

In [54]:
valid_status_codes = list(status['Code'])

In [55]:
status_dict = dict(zip(status['Code'], status['Status']))

In [56]:
pending = ['2A', '2B', '8A', '8B', '8D']
benefit = ['5C', '5D']
expedited_admin_rein = ['8F', '8G', '8H', '8I', '11', '16']
problematic = pending + benefit + expedited_admin_rein

In [57]:
problematic_count = 0
valid = df[removals]['Status'].str.upper().isin(valid_status_codes)
valid_count = sum(valid)
invalid = ~df[removals]['Status'].str.upper().isin(valid_status_codes)
invalid_count = sum(invalid)

print(f'ICE Air - All removals')
print()
print(f'ICE status codes for passengers on direct removal flights.')
print(f'(Statuses which may raise due process concerns starred)')
print()
print(f'{valid_count} valid values.')
print(f'{invalid_count} invalid values.')
print()
for c in valid_status_codes:
    
    try:
        count = status_count[c]
    except KeyError:
        count = 0
    
    if c in problematic:
        problematic_count = problematic_count + count
        print(f'[{c}]: *{count}* ({status_dict[c]})')
    else:
        print(f'[{c}]: {count} ({status_dict[c]})')
        
assert sum(df[removals]['Status'].isin(problematic)) == problematic_count
print()
print(f'{problematic_count} total problematic cases.')
print(f"{sum(df.loc[removals, 'Status'].isin(pending))} with pending appeals.")
print(f"{sum(df.loc[removals, 'Status'].isin(benefit))} with benefit blocking deportation.")
print(f"{sum(df.loc[removals, 'Status'].isin(expedited_admin_rein))} under expedited, administrative, reinstatement of removal.")

ICE Air - All removals

ICE status codes for passengers on direct removal flights.
(Statuses which may raise due process concerns starred)

839344 valid values.
394568 invalid values.

[1A]: 55104 (Voluntary Departure – Un-Expired and Un-Extended Voluntary Departure.)
[1B]: 284 (Voluntary Departure - Extended Departure Period)
[1C]: 8 (Exipred Voluntary Departure Period - Referred to Investigation)
[2A]: *2223* (Deportable – Under Adjudication by IJ)
[2B]: *96* (Deportable – Under Adjudication by BIA)
[3]: 20572 (Deportable – Administratively Final Order)
[5A]: 14 (Referred for Investigation - No Show for Hearing - No Final O...)
[5B]: 797 (Removable – ICE Fugitive)
[5C]: *16* (Relief Granted – Withholding of Deportation/Removal)
[5D]: *86* (Final Order of Deportation/Removal – Deferred Action Granted.)
[5E]: 32 (Relief Granted – Extended Voluntary Departure)
[5F]: 57 (Unable to Obtain Travel Document)
[8A]: *1489* (Excludable/Inadmissible – Hearing Not Commenced)
[8B]: *4080* (Excluda

In [12]:
sum(df[removals]['Status'].isin(expedited_admin_rein))

627694

In [14]:
invalid_status = df[~df['Status'].isin(valid_status_codes)]
invalid_status_counts = invalid_status['Status'].value_counts()
# bad_statuses = list(status_counts[status_counts > 0].index)

In [15]:
invalid_status_counts.head(25)

REMOVAL           231309
RM                164378
NO STATUS          50831
NTA                20931
TRANSFER - F/O     19507
TRANSFER - NTA      9795
REINSTATE           6819
ER-TRANSFER         5917
STAGER              4771
NTA - TRANSFER      3036
F/O - TRANSFER      2620
TRANSFER            2174
REINST              1800
TRANSFER-NTA        1468
WD                  1030
FINAL ORDER          909
JUVI                 845
1/15/1900            747
NTA-TRANSFER         680
WA/NTA               666
NTA-TRFD             663
V/R                  635
DEPORT               584
REIN                 569
RE-INSTATE           484
Name: Status, dtype: int64

## Status by FY for removals & transfers

In [16]:
status_fy = df[removals].drop_duplicates(subset='AlienMasterID').resample('AS-OCT', on='MissionDate')['Status'].value_counts().unstack()

In [17]:
total = df[removals].drop_duplicates(subset='AlienMasterID').resample('AS-OCT', on='MissionDate')['AlienMasterID'].nunique()

In [18]:
status_fy[expedited_admin_rein]

Status,8F,8G,8H,8I,11,16
MissionDate,,,,,,
2010-10-01,9899.0,1.0,NaN,NaN,174.0,6484.0
2011-10-01,22038.0,NaN,NaN,NaN,92.0,13747.0
2012-10-01,52132.0,462.0,9.0,5.0,2298.0,45536.0
2013-10-01,53894.0,2556.0,164.0,33.0,3008.0,50542.0
2014-10-01,32056.0,4049.0,61.0,80.0,3832.0,40019.0
2015-10-01,29940.0,10335.0,246.0,128.0,3741.0,42121.0
2016-10-01,26146.0,13062.0,1264.0,214.0,2913.0,41956.0
2017-10-01,32839.0,12492.0,349.0,276.0,2652.0,45714.0
2018-10-01,6290.0,2686.0,38.0,45.0,399.0,8677.0


In [19]:
# status_fy[valid_status_codes].to_csv('../output/valid_status_by_year.csv')

In [20]:
all_others = status_fy[status_fy.columns[~status_fy.columns.isin(valid_status_codes)]].sum(axis=1)
all_others.name = 'all_others'

In [21]:
status_table = pd.concat([status_fy[valid_status_codes], all_others], axis=1).T

In [22]:
status_table

MissionDate,2010-10-01 00:00:00,2011-10-01 00:00:00,2012-10-01 00:00:00,2013-10-01 00:00:00,2014-10-01 00:00:00,2015-10-01 00:00:00,2016-10-01 00:00:00,2017-10-01 00:00:00,2018-10-01 00:00:00
1A,22425.0,22071.0,9738.0,513.0,264.0,36.0,23.0,27.0,7.0
1B,NaN,NaN,112.0,93.0,72.0,1.0,1.0,4.0,1.0
1C,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN,NaN
2A,NaN,NaN,994.0,233.0,301.0,106.0,217.0,325.0,47.0
2B,NaN,NaN,15.0,13.0,11.0,15.0,24.0,15.0,3.0
3,1149.0,555.0,1281.0,1737.0,2645.0,3354.0,4033.0,4953.0,865.0
5A,NaN,NaN,2.0,4.0,3.0,NaN,2.0,2.0,1.0
5B,NaN,NaN,68.0,241.0,141.0,65.0,133.0,137.0,12.0
5C,NaN,NaN,1.0,4.0,4.0,1.0,2.0,3.0,1.0
5D,NaN,NaN,16.0,18.0,8.0,4.0,19.0,17.0,4.0


In [23]:
# status_table.to_csv('../output/removal_status_by_year.csv')
# (status_table/total).to_csv('../output/removal_status_by_year_percent.csv')

## Status for specific airport

In [24]:
airport = 'KBFI'

In [25]:
airport_name = airport_dict[airport]
pickups = df['PULOC'] == airport
dropoffs = df['DropLoc'] == airport
airport_df = df[pickups|dropoffs]

# Status for specific nationality

In [58]:
nationality = 'CAMBODIA'

errormsg = f'{nationality} not represented in dataset'
assert nationality.upper() in df['CountryOfCitizenship'].str.upper().unique(), errormsg

nationality_df = df[df['CountryOfCitizenship'] == nationality.upper()]

removals = nationality_df['R-T'] == 'R'
transfers = nationality_df['R-T'] == 'T'

In [59]:
status_count = nationality_df[removals].drop_duplicates(subset='AlienMasterID')['Status'].str.upper().value_counts(dropna=False)
count = 0
problematic_count = 0
valid = nationality_df[removals]['Status'].str.upper().isin(valid_status_codes)
valid_count = sum(valid)
invalid = ~nationality_df[removals]['Status'].str.upper().isin(valid_status_codes)
invalid_count = sum(invalid)

print(f'ICE Air - {nationality} removals')
print()
print(f'ICE status codes for passengers on direct removal flights.')
print(f'(Statuses which may raise due process concerns starred)')
print()
print(f'{valid_count} valid values.')
print(f'{invalid_count} invalid values.')
print()
for c in valid_status_codes:
    
    try:
        count = status_count[c]
    except KeyError:
        count = 0
    
    if c in problematic:
        problematic_count = problematic_count + count
        print(f'[{c}]: *{count}* ({status_dict[c]})')
    else:
        print(f'[{c}]: {count} ({status_dict[c]})')
        
assert sum(nationality_df[removals]['Status'].isin(problematic)) == problematic_count
print()
print(f'{problematic_count} total problematic cases.')
print(f"{sum(nationality_df.loc[removals, 'Status'].isin(pending))} with pending appeals.")
print(f"{sum(nationality_df.loc[removals, 'Status'].isin(benefit))} with benefit blocking deportation.")
print(f"{sum(nationality_df.loc[removals, 'Status'].isin(expedited_admin_rein))} under expedited, administrative, reinstatement of removal.")

ICE Air - CAMBODIA removals

ICE status codes for passengers on direct removal flights.
(Statuses which may raise due process concerns starred)

120 valid values.
0 invalid values.

[1A]: 0 (Voluntary Departure – Un-Expired and Un-Extended Voluntary Departure.)
[1B]: 0 (Voluntary Departure - Extended Departure Period)
[1C]: 0 (Exipred Voluntary Departure Period - Referred to Investigation)
[2A]: *0* (Deportable – Under Adjudication by IJ)
[2B]: *0* (Deportable – Under Adjudication by BIA)
[3]: 105 (Deportable – Administratively Final Order)
[5A]: 0 (Referred for Investigation - No Show for Hearing - No Final O...)
[5B]: 2 (Removable – ICE Fugitive)
[5C]: *0* (Relief Granted – Withholding of Deportation/Removal)
[5D]: *0* (Final Order of Deportation/Removal – Deferred Action Granted.)
[5E]: 0 (Relief Granted – Extended Voluntary Departure)
[5F]: 5 (Unable to Obtain Travel Document)
[8A]: *0* (Excludable/Inadmissible – Hearing Not Commenced)
[8B]: *0* (Excludable/Inadmissible – Under Adj

In [60]:
pending = ['2A', '2B', '8A', '8B', '8D']
benefit = ['5C', '5D']
expedited_admin_rein = ['8F', '8G', '8H', '8I', '11', '16']

In [61]:
sum(nationality_df.loc[removals, 'Status'].isin(pending))

0

In [62]:
sum(nationality_df.loc[removals, 'Status'].isin(benefit))

0

In [63]:
sum(nationality_df.loc[removals, 'Status'].isin(expedited_admin_rein))

1

In [65]:
invalid_status = nationality_df[~nationality_df['Status'].isin(valid_status_codes)]
invalid_status_counts = invalid_status['Status'].value_counts()
# bad_statuses = list(status_counts[status_counts > 0].index)

In [67]:
invalid_status_counts.head(10)

TRANSFER - F/O    143
TRANSFER           67
TRANSFER - NTA     40
NO STATUS          39
REMOVAL            33
NTA                10
F/O - TRANSFER      7
STAGING             4
INTERVIEW           2
RR - TRANSFER       2
Name: Status, dtype: int64